In [ ]:
We need the following tables and fields:

users:
- id
- timestamp

activities:
- user_id
- timestamp

In this exercise we are doing the following:

1. We cohort users by their signup month. This is the users table below
2. For each cohort, we show the population (how many people) in that cohort in the second column
3. An activity is an action user did to the site (this could be simply view the page, or as specific as making a purchase).


In [ ]:
#this is in postgres - date_trunc is a postgres function

1. to get the month when customers originally signed up (-- (user_id, cohort_month), each)

with original_cohort_months as (
  select
    date_trunc('month', U.created_at)::date as cohort_month,
    id as user_id
  from users U
  order by 1, 2 #here 1 means: cohort_month and 2 means id
),

output will be like:

cohort_month           id

2017-12-01             2345  #the id 2345 wont get repeated in any other months
2018-01-01             6578 
2018-02-01             9876
2018-03-01             1234
2018-04-01             1123
2018-05-01             5434
2018-06-01             3423
2018-07-01             6766
2018-08-01             9883
2018-09-01             4343
2018-10-01             4442

etc.,


the output of date_trunc used in 1 looks like in the following format:
    
date_trunc('month',created_at)::date 
// 2019-12-01


More examples:
    
assume input as:
created_at = '2019-12-16 18:28:13'


date_trunc('month',created_at)::date 
// 2019-12-01

date_trunc('year',created_at)::date 
// 2019-01-01

date_trunc('day',created_at)::date 
// 2019-12-16


if you dont use ::date then date_trunc looks like as follows:
            
date_trunc('day',created_at)
// 2019-12-16 00:00:00
        
        

In [ ]:

2. -- (user_id, month_number): user X has activity in month number X

with user_activities as (
  select
    A.user_id as userid, (select extract(year from age(A.created_at, C.cohort_month)) * 12 +
extract(month from age(A.created_at, C.cohort_month))) as month_number
  from activities A
  left join original_cohort_months C ON A.user_id = C.user_id
  group by 1, 2 #here 1 means userid and 2 means month_number
),



So the output will be like

userid  month_number
Tyler     0
Tyler     1
Toby      3
Toma      4
Chris     1
Chris     2
Chris     3
Goalk     3
Goalk     4
Goalk     5

etc., 

In [ ]:


3. -- Selecting the numebr of registered/signedup users per month (cohort_month, size)

with cohort_size as (
  select cohort_month, count(1) as num_users_signedup
  from original_cohort_months
  group by 1
  order by 1
),

output will be like:

cohort_month      num_users_signedup
2017-12-01                   3234
2018-01-01                   5446
2018-02-01                   6574
2018-03-01                   7645
2018-04-01                   7676
2018-05-01                   8765
2018-06-01                   8112
2018-07-01                   6542
2018-08-01                   4456
2018-09-01                   5678
2018-10-01                   8990
2018-11-01                   9832

In [ ]:

4. -- (cohort_month, month_number, cnt)

with B as (
  select
    C.cohort_month as cohort_month,
    A.month_number as month_number,
    count(1) as num_users_active
  from user_activities A
  left join original_cohort_months C ON A.user_id = C.user_id
  group by 1, 2 #here 1 means C.cohort_month and 2 means A.month_number
),

output looks like:

cohort_month    month_number    num_users_active
2017-12-01       00             1343
2017-12-01       01             2343
2017-12-01       02             1333
2017-12-01       03             1232
2017-12-01       04             2000

2018-01-01       00             3454
2018-01-01       01             4444

2018-02-01       00             5545
2018-02-01       01             6565


In [ ]:

5. -- our final value: (cohort_month, size, month_number, percentage)

select
  B.cohort_month as cohort_month,
  S.num_users_signedup as total_users,
  B.month_number as active_month_number,
  B.num_users_active::float * 100 / S.num_users_signedup as percentage_active_users
from B
left join cohort_size S ON B.cohort_month = S.cohort_month
order by 1, 3
where B.cohort_month IS NOT NULL;


cohort_month    total_users    active_month_number    percentage_active_users
2017-12-01      3234                  00              (1343/3234)*100
2017-12-01      3234                  01              (2343/3234)*100
2017-12-01      3234                  02              (1333/3234)*100
2017-12-01      3234                  03              (1232/3234)*100
2017-12-01      3234                  04              (2000/3234)*100

2018-01-01      5446                  00              (3454/5446)*100
2018-01-01      5446                  01              (4444/5446)*100

2018-02-01      6574                  00              (5545/6574)*100
2018-02-01      6574                  01              (6565/6574)*100


In [ ]:
Lets put this all together



with original_cohort_months as (
  select
    date_trunc('month', U.created_at)::date as cohort_month,
    id as user_id
  from users U
  order by 1, 2 #here 1 means: cohort_month and 2 means id
),

user_activities as (
  select
    A.user_id as userid, (select extract(year from age(A.created_at, C.cohort_month)) * 12 +
extract(month from age(A.created_at, C.cohort_month))) as month_number
  from activities A
  left join original_cohort_months C ON A.user_id = C.user_id
  group by 1, 2 #here 1 means userid and 2 means month_number
),


cohort_size as (
  select cohort_month, count(1) as num_users_signedup
  from original_cohort_months
  group by 1
  order by 1
),


B as (
  select
    C.cohort_month as cohort_month,
    A.month_number as month_number,
    count(1) as num_users_active
  from user_activities A
  left join original_cohort_months C ON A.user_id = C.user_id
  group by 1, 2 #here 1 means C.cohort_month and 2 means A.month_number
),


select
  B.cohort_month as cohort_month,
  S.num_users_signedup as total_users,
  B.month_number as active_month_number,
  B.num_users_active::float * 100 / S.num_users_signedup as percentage_active_users
from B
left join cohort_size S ON B.cohort_month = S.cohort_month
order by 1, 3
where B.cohort_month IS NOT NULL;



    
    


In [ ]:
custom MONTH_DIFF function (inspired by https://gist.github.com/nvquanghuy/bd0fda7b88e5b2fd8e46e047e391d25f)

CREATE FUNCTION MONTH_DIFF (t_start timestamp, t_end timestamp)
	RETURNS integer
	AS $$
	SELECT
		(12 * extract('years' FROM a.i) + extract('months' FROM a.i))::integer
	FROM (
		values(justify_interval($2 - $1))) AS a (i)
$$
LANGUAGE SQL
IMMUTABLE
	RETURNS NULL ON NULL INPUT;
    
    